<img src="img/Logo_OpenClassrooms.png" width=100 />

# OpenClassrooms Projet 5: Segmentez des clients d'un site e-commerce
# Analyse Exploratoire

Olist souhaite que vous fournissiez à ses équipes d'e-commerce une segmentation des clients qu’elles pourront utiliser au quotidien pour leurs campagnes de communication.

Votre objectif est de comprendre les différents types d’utilisateurs grâce à leur comportement et à leurs données personnelles.

Vous devrez fournir à l’équipe marketing une description actionable de votre segmentation et de sa logique sous-jacente pour une utilisation optimale, ainsi qu’une proposition de contrat de maintenance basée sur une analyse de la stabilité des segments au cours du temps.

### Les données
Pour cette mission, Olist vous fournit une base de données anonymisée comportant des informations sur l’historique de commandes, les produits achetés, les commentaires de satisfaction, et la localisation des clients depuis janvier 2017.

### Votre mission
Votre mission est d’aider les équipes d’Olist à comprendre les différents types d'utilisateurs. Vous utiliserez donc des méthodes non supervisées pour regrouper ensemble des clients de profils similaires. Ces catégories pourront être utilisées par l’équipe marketing pour mieux communiquer.

Pour des raisons de confidentialité, Olist ne fournit pas beaucoup de données, à vous de fouiller dans celles dont vous disposez et de créer les meilleures features pour les exploiter.

Enfin, votre client, Olist, a spécifié sa demande ainsi :

- La segmentation proposée doit être exploitable et facile d’utilisation pour l’équipe marketing.
- Vous évaluerez la fréquence à laquelle la segmentation doit être mise à jour, afin de pouvoir effectuer un devis de contrat de maintenance.
- Le code fourni doit respecter la convention PEP8, pour être utilisable par Olist.


# Sommaire
### [Bibliothèques](#1_bibli)
### [Fonctions](#1_funcs)
### [Données](#1_donnees)
### [Création de la base](#1_creation_base)


<a id='1_bibli'></a>
# Import de bibliothèques 📚

In [1]:
import pandas as pd
pd.set_option("mode.chained_assignment", None)

<a id='1_funcs'></a>

# Fonctions ⚙️

<a id='get_orders'></a>

In [2]:
def get_orders(client_unique_id):
    """ A partir d'un id client unique la fonction retourne la liste des commandes correspondantes,
    La table des commandes doit au préalable être filtrée en fonction de la date virtuelle de l'étude
    Args:
        client_unique_id
    """
    
    # Avec l'id unique du client on récupère les id clients qui sont liés à chaque commande
    ids_client = customers[customers['customer_unique_id'] == client_unique_id].customer_id.values
    # On peut ensuite récupérer les id des commandes correspondant à ces ids clients
    orders_client = orders[orders.customer_id.isin(ids_client)]
    # On récupère juste les ids
    ids_orders_client = orders_client.index.values
    return ids_orders_client

[Retour au code](#get_orders_back)

<a id='1_donnees'></a>

# Données 🎁

<img src="img/structure.png" width=500 />

In [3]:
orders = pd.read_csv('data/olist_orders_dataset.csv', parse_dates=['order_purchase_timestamp'], index_col='order_id')
order_items = pd.read_csv('data/olist_order_items_dataset.csv')
products = pd.read_csv('data/olist_products_dataset.csv')
customers = pd.read_csv('data/olist_customers_dataset.csv')
reviews = pd.read_csv('data/olist_order_items_dataset.csv')
geolocs = pd.read_csv('data/olist_geolocation_dataset.csv')
payments = pd.read_csv('data/olist_order_payments_dataset.csv')

<a id='1_creation_base'></a>

# Création d'une base répondant au besoin ✅
La base doit avoir pour index les ids des clients avec des variables en relation avec son comportement.

Variables retenues:
- [Nombre de commandes](#nb_commandes)
- [Temps depuis la première commande](#tps_last_order)
- Montants dépensés
- Types de produits achetés
- Nombre de produits par commande
- Temps depuis la dernière commande
- Position
- Nombre de reviews postés

### Important:
Pour le bon fonctionnement du projet, les fonctions développées pour la constitution de la base prendront en paramètre la date à laquelle on veut se fixer virtuellement. En effet on cherchera plus tard dans ce projet à déterminer la fréquence de mise à jour de la base d'entrainement du modèle.
On prendra toujours dans ce notebook la dernière date de la base de donnée.

In [4]:
virtual_date = orders.order_purchase_timestamp.max()

In [5]:
orders.order_purchase_timestamp.max()

Timestamp('2018-10-17 17:30:18')

<a id='nb_commandes'></a>

## Nombre de commandes par client 🥰

Cette variable est la plus importante car va nous permettre de connaitre la taille de notre dataset. En effet, difficile de classer des clients n'ayant fait qu'une commande

Regardons comment fonctionnent les id des clients

In [7]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [8]:
len(customers.customer_unique_id.unique())

96096

In [9]:
len(customers.customer_id.unique())

99441

In [10]:
orders.columns

Index(['customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

On filtre ici la date virtuelle pour garder un trace mais cette commande est neutre

In [11]:
orders = orders[orders.order_purchase_timestamp <= virtual_date]

In [12]:
orders = orders.merge(customers[['customer_id', 'customer_unique_id']], how='left').set_axis(orders.index)

On trie par date pour conserver un ordre des commandes logique

In [13]:
orders.sort_values('order_purchase_timestamp', inplace=True)

In [14]:
(orders.groupby('customer_unique_id').count().customer_id > 1).value_counts()

False    93099
True      2997
Name: customer_id, dtype: int64

### Il n'y a dans la base clients que 2997 clients ayant fait plus d'un commande

In [15]:
data = pd.DataFrame(orders.groupby('customer_unique_id').count().customer_id)

In [16]:
data.columns = ['nb_orders']

Nous avons besoin de clients ayant plus d'une commande pour commencer à comprendre leur comportement. Cela réduit considérablement la taille de la base

In [17]:
data = data[data.nb_orders > 1]

<a id='get_orders_back'></a>
### Pour les autres variables nous avons besoin des ids des commandes correspondant à chaque client

Voir la [fonction](#get_orders)

In [18]:
get_orders.__doc__

"A partir d'un id_client_unique la fonction retourne la liste des commandes correspondantes, La table des commandes doit au préalable être filtrée en fonction de la date virtuelle de la commande"

In [19]:
data['orders_ids'] = data.index.map(get_orders)

In [20]:
data.head()

,nb_orders,orders_ids
customer_unique_id,,
00172711b30d52eea8b313a7f2cced02,2,"[bb874c45df1a3c97842d52f31efee99a, c306eca42d3..."
004288347e5e88a27ded2bb23747066c,2,"[a61d617fbe5bd006e40d3a0988fc844b, 08204559beb..."
004b45ec5c64187465168251cd1c9c2f,2,"[90ae229a4addcfead792e2564554f09c, 9392c5e7288..."
0058f300f57d7b93c477a131a59b36c3,2,"[2cfc79d9582e9135c0a9b61fa60e6b21, 81a93b2fa39..."
00a39521eb40f7012db50455bf083460,2,"[7d32c87acba91ed87ebd98310fe1c54d, cea3e6c11eb..."


<a id='tps_last_order'></a>
## Temps depuis la première commande ⏲️
Pour représenter l'ancienneté du client. On aura besoin ici de la date virtuelle

In [21]:
def get_nb_days(ids_order, virtual_date=virtual_date):
    """Cette fonction calcule le nombre de jours écoulés depuis la première et la dernière commande en prenant pour référence la date virtuelle de l'étude"""
    date_first_order = orders.loc[ids_order[0], 
    date_last_order = ids_order[-1]
    
    
    

SyntaxError: invalid syntax (<ipython-input-21-ced8831d4958>, line 4)

In [ ]:
orders[orders.order_id == data.iloc[0].orders_ids[0]].order_purchase_timestamp.iloc[0]

In [ ]:
orders.loc['bb874c45df1a3c97842d52f31efee99a', 'order_purchase_timestamp']

In [ ]:
orders

In [ ]:
order_items

In [ ]:
len(order_items.order_id.unique())

In [ ]:
order_items.order_item_id.value_counts()

Seulement 9803 commandes ont au moins deux items